# 🚀 Install, Import, and Log in

### Step 0️⃣: Install W&B

In [13]:
%%capture
#!pip install wandb
!pip install pandas

### Step 1️⃣: Import W&B and Login

In [14]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import os
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
import pickle5 as pickle

from sklearn.model_selection import train_test_split 

#%matplotlib inline

In [15]:
import wandb
from wandb.keras import WandbCallback

#wandb.init()  # defaults are over-ridden during the sweep


wandb.login()

True

In [16]:

fluxData_df =pd.read_hdf("data/fluxData_df.h5", key="fluxData_df")
zernikeData_df =pd.read_hdf("data/zernikeData_df.h5", key="zernikeData_df")

Read data in

# 👩‍🍳 Prepare Dataset

In [17]:
# Prepare the training dataset
X = fluxData_df
y = zernikeData_df
x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42)


# Build simple Model

In [18]:
AoModel = keras.Sequential([
     keras.layers.InputLayer(19, name="digits"),
     keras.layers.Dense(2000, activation="relu"),
     keras.layers.Dense(1050, activation="relu"),
     keras.layers.Dense(100, activation="relu"),
     keras.layers.Dense(9, activation="linear", name="predictions"),

])
AoModel.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 2000)              40000     
_________________________________________________________________
dense_4 (Dense)              (None, 1050)              2101050   
_________________________________________________________________
dense_5 (Dense)              (None, 100)               105100    
_________________________________________________________________
predictions (Dense)          (None, 9)                 909       
Total params: 2,247,059
Trainable params: 2,247,059
Non-trainable params: 0
_________________________________________________________________


In [19]:
batch_size = 128
epochs = 200


    # optimizer='sgd',
    # loss='mse',
    # metrics=[tf.keras.metrics.RootMeanSquaredError()])



AoModel.compile(loss= keras.losses.MeanSquaredError(), optimizer="adam", metrics= [tf.keras.metrics.RootMeanSquaredError()])
#AoModel.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose = 2,  callbacks=[WandbCallback()])

#AoModel.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=[WandbCallback()])

# 🧠 Define the wandb sweep

In [20]:
os.environ["WANDB_NOTEBOOK_NAME"] = "AoKerasStudio.ipynb"

sweep_config = {
    "method": "random", # try grid or random

    "parameters": {
        "booster": {
            "values": ["gbtree"]
        },
        "max_depth": {
      "min": 6,
      "max": 10
        },

    "learning_rate" :{
        "values": [0.04954]
        },        
    "subsample": {
        "values": [0.4372]
    },
    "colsample_bynode": {
        "values": [0.7597]
    },    
    "n_estimators": {
        "values": [3000, 10000, 20000]
    },
    "reg_alpha": {
        "values": [0.1668]
    },    
    "reg_lambda": {
        "values": [0.5222]
    }                
    }
}

In [21]:
## Define training loop

In [22]:
def train():
  config_defaults = {
  "booster": "gbtree",
  "max_depth": 3,
  "learning_rate": 0.1,
  "subsample": 1,
  "seed": 117,
  "test_size": 0.33,
}
  wandb.init(config=config_defaults)  # defaults are over-ridden during the sweep
  #config = wandb.config
  AoModel.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose = 2,  callbacks=[WandbCallback()])
  wandb.log()

## Initialize sweep

In [23]:
sweep_id = wandb.sweep(sweep_config, entity = "uwe-sterr",project="KerasAoSweep")
wandb.run

Create sweep with ID: xzrv0sqq
Sweep URL: https://wandb.ai/uwe-sterr/KerasAoSweep/sweeps/xzrv0sqq


## Run sweep

In [24]:
wandb.agent(sweep_id, train, count=2)

wandb: Agent Starting Run: 54mf4d6o with config:
wandb: 	booster: gbtree
wandb: 	colsample_bynode: 0.7597
wandb: 	learning_rate: 0.04954
wandb: 	max_depth: 8
wandb: 	n_estimators: 20000
wandb: 	reg_alpha: 0.1668
wandb: 	reg_lambda: 0.5222
wandb: 	subsample: 0.4372


Run 54mf4d6o errored: ValueError('Tensor("Adam_1/dense_3/kernel/m:0", shape=(), dtype=resource) must be from the same graph as Tensor("dense_3/kernel:0", shape=(), dtype=resource).')
wandb: ERROR Run 54mf4d6o errored: ValueError('Tensor("Adam_1/dense_3/kernel/m:0", shape=(), dtype=resource) must be from the same graph as Tensor("dense_3/kernel:0", shape=(), dtype=resource).')
wandb: Agent Starting Run: igpz422o with config:
wandb: 	booster: gbtree
wandb: 	colsample_bynode: 0.7597
wandb: 	learning_rate: 0.04954
wandb: 	max_depth: 8
wandb: 	n_estimators: 10000
wandb: 	reg_alpha: 0.1668
wandb: 	reg_lambda: 0.5222
wandb: 	subsample: 0.4372


Run igpz422o errored: ValueError('Tensor("Adam_1/dense_3/kernel/m:0", shape=(), dtype=resource) must be from the same graph as Tensor("dense_3/kernel:0", shape=(), dtype=resource).')
wandb: ERROR Run igpz422o errored: ValueError('Tensor("Adam_1/dense_3/kernel/m:0", shape=(), dtype=resource) must be from the same graph as Tensor("dense_3/kernel:0", shape=(), dtype=resource).')
